In [1]:
import requests
from datetime import datetime
import traceback
import time
import json
import logging
import pandas as pd

## Set up Logging

In [2]:
logger = logging.getLogger("reddit scraper")
logger.setLevel("DEBUG")
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

## Global Variables

In [3]:
subreddit = "dataengineering"

In [4]:
base_url = "https://api.pushshift.io/reddit/submission/search?\
&subreddit={subreddit}\
&limit={limit}\
&sort={sort}\
&sort_type={sort_type}\
&before={end_timestamp}\
&{start_timestamp}"

In [5]:
excluded_columns = [
    "all_awardings",
    "author_flair_richtext",
    "awarders",
    "gildings",
    "link_flair_richtext",
    "treatment_tags",
    "wls",
    "pwls",
    "preview",
    "poll_data",
    "media_metadata",
    "crosspost_parent_list",
    "media",
    "media_embed",
    "secure_media",
    "secure_media_embed",
    "collections",
    "gallery_data",
    "steward_reports",
    "approved_at_utc",
    "banned_at_utc",
    "suggested_sort",
    "view_count",
]

## helper function to get posts

In [6]:
def get_posts(json_objects):
    posts = []
    for post in json_objects:
        filtered_posts = {k:v for (k,v) in post.items() if k not in excluded_columns}
        posts.append(filtered_posts)
    return posts

## Get all subreddit posts

In [7]:
def get_all_subreddit_posts(pushshift_args, full_refresh = True):
    all_posts = []
    start_time = datetime.utcnow()
    previous_epoch = int(start_time.timestamp())
    if not full_refresh:
        pushshift_args["start_timestamp"] = None
    while True: # loop through requests until empty
        pushshift_args["end_timestamp"] = previous_epoch
        request_url = base_url.format(**pushshift_args)
        json_text = requests.get(request_url) # get reddit data via request from pushshift
        time.sleep(1)
        try: # parse request to json
            json_request = json_text.json()
        except json.decoder.JSONDecodeError as e:
            logger.warning("Unable to Decode json request:", e)
            time.sleep(1)
            continue
            
        json_objects = json_request.get('data', [])
        if not json_objects: # check if result set is empty
            logger.warning("Empty json request")
            break
            
        previous_epoch = json_objects[-1]['created_utc'] - 1
        total_posts = len(json_objects)
            
        posts = get_posts(json_objects) # get posts from request
        all_posts.extend(posts)
        logger.info(f"Processed {len(all_posts)} posts")

    logger.info(f"Saved {len(all_posts)} posts")
    return all_posts

In [8]:
pushshift_args = {
    'subreddit': 'dataengineering',
    'limit': 1000,
    'sort': 'desc',
    'sort_type': 'created_utc',
    'end_timestamp': None,
    'start_timestamp': None
}

In [9]:
all_posts = get_all_subreddit_posts(pushshift_args)

2021-12-28 21:00:31,908 - reddit scraper - INFO - Processed 100 posts
2021-12-28 21:00:33,754 - reddit scraper - INFO - Processed 200 posts
2021-12-28 21:00:35,828 - reddit scraper - INFO - Processed 300 posts
2021-12-28 21:00:37,279 - reddit scraper - INFO - Processed 400 posts
2021-12-28 21:00:39,488 - reddit scraper - INFO - Processed 500 posts
2021-12-28 21:00:41,150 - reddit scraper - INFO - Processed 600 posts
2021-12-28 21:00:43,055 - reddit scraper - INFO - Processed 700 posts
2021-12-28 21:00:44,490 - reddit scraper - INFO - Processed 800 posts
2021-12-28 21:00:46,513 - reddit scraper - INFO - Processed 900 posts
2021-12-28 21:00:48,491 - reddit scraper - INFO - Processed 1000 posts
2021-12-28 21:00:49,801 - reddit scraper - INFO - Processed 1100 posts
2021-12-28 21:00:54,382 - reddit scraper - INFO - Processed 1200 posts
2021-12-28 21:00:56,233 - reddit scraper - INFO - Processed 1300 posts
2021-12-28 21:00:58,086 - reddit scraper - INFO - Processed 1400 posts
2021-12-28 21:0

### Create dataframe from results

In [10]:
df = pd.DataFrame(all_posts)

### Drop Duplicates

In [11]:
df_deduped = df.drop_duplicates(['author', 'title', 'created_utc'])

### Save Dataframe to CSV

In [12]:
df_deduped.to_csv("reddit_de_posts_filtered.csv", index = False)

## Save results to Postgres Database

### Connect to Postgres Database

In [13]:
import psycopg2
import os
from dotenv import load_dotenv

load_dotenv("../postgres/.env")

True

In [14]:
PG_USER = os.getenv('POSTGRES_USER')
PG_PW = os.getenv('POSTGRES_PASSWORD')
PG_DB = os.getenv('POSTGRES_DB')

In [15]:
postgres_conn_args = {
    "host": "localhost",
    "database": PG_DB,
    "user": PG_USER,
    "password": PG_PW,
    "port": 5432
}

In [16]:
pg_conn = psycopg2.connect(**postgres_conn_args)

### Create table

In [17]:
CREATE_TABLE_SQL = """
CREATE TABLE IF NOT EXISTS raw_data.reddit_de (
    allow_live_comments bool,
    author text NOT NULL,
    author_flair_css_class text,
    author_flair_text text,
    author_flair_type text,
    author_fullname text,
    author_is_blocked bool,
    author_patreon_flair bool,
    author_premium bool,
    can_mod_post bool,
    contest_mode bool,
    created_utc bigint NOT NULL,
    domain text,
    full_link text NOT NULL,
    id char(6) UNIQUE NOT NULL,
    is_created_from_ads_ui bool,
    is_crosspostable bool,
    is_meta bool,
    is_original_content bool,
    is_reddit_media_domain bool,
    is_robot_indexable bool,
    is_self bool,
    is_video bool,
    link_flair_background_color varchar(7),
    link_flair_template_id text,
    link_flair_text text,
    link_flair_text_color text,
    link_flair_type text,
    locked bool,
    media_only bool,
    no_follow bool,
    num_comments int,
    num_crossposts decimal(3,1),
    over_18 bool,
    parent_whitelist_status text,
    permalink text NOT NULL,
    pinned bool,
    retrieved_on bigint NOT NULL,
    score int,
    selftext text,
    send_replies bool,
    spoiler bool,
    stickied bool,
    subreddit text,
    subreddit_id text,
    subreddit_subscribers decimal(7,1),
    subreddit_type text,
    thumbnail text,
    title text NOT NULL,
    total_awards_received decimal(4,1),
    upvote_ratio decimal(3,2),
    url text NOT NULL,
    whitelist_status text,
    post_hint text,
    thumbnail_height decimal(4,1),
    thumbnail_width decimal(4,1),
    url_overridden_by_dest text,
    removed_by_category text,
    author_flair_background_color text,
    author_flair_template_id text,
    author_flair_text_color text,
    author_cakeday bool,
    crosspost_parent char(9),
    edited decimal(11,1),
    banned_by text,
    is_gallery bool,
    distinguished text,
    removed_by text,
    updated_utc decimal(11,1),
    og_description text,
    og_title text,
    gilded decimal(2,1),
    rte_mode text,
    author_id text,
    brand_safe bool,
    author_created_utc decimal(11,1)
);
"""

In [19]:
with pg_conn:
    with pg_conn.cursor() as pg_cur:
            pg_cur.execute(CREATE_TABLE_SQL)

### Copy Contents to table

In [20]:
COPY_SQL = """
COPY raw_data.reddit_de {columns}
FROM STDIN
WITH CSV HEADER
DELIMITER as ','
"""

In [21]:
column_names = f"({', '.join(list(df_deduped.columns))})"

In [22]:
with pg_conn:
    with pg_conn.cursor() as pg_cur:
        with open("reddit_de_posts_filtered.csv",  'rb') as tmp_file:
            pg_cur.copy_expert(COPY_SQL.format(columns=column_names), tmp_file)

### Clean Up

In [23]:
with pg_conn:
    with pg_conn.cursor() as pg_cur:
        pg_cur.execute("ANALYZE")

In [24]:
pg_conn.close()

In [184]:
!rm -f 'reddit_de_posts_filtered.csv'

In [218]:
df["is_reddit_media_domain"].value_counts()

False    8921
True       87
Name: is_reddit_media_domain, dtype: int64

In [219]:
df["is_robot_indexable"].value_counts()

True     7667
False    1169
Name: is_robot_indexable, dtype: int64

In [234]:
df[df["is_robot_indexable"]==True][df[df["is_robot_indexable"]==True].author.str.startswith("[")]

,allow_live_comments,author,author_flair_css_class,author_flair_text,author_flair_type,author_fullname,author_is_blocked,author_patreon_flair,author_premium,can_mod_post,...,distinguished,removed_by,updated_utc,og_description,og_title,gilded,rte_mode,author_id,brand_safe,author_created_utc
7330,False,[deleted],None,None,None,None,None,None,None,False,...,None,None,NaN,None,None,NaN,None,None,None,NaN


In [235]:
df[df["media_only"] == True]

,allow_live_comments,author,author_flair_css_class,author_flair_text,author_flair_type,author_fullname,author_is_blocked,author_patreon_flair,author_premium,can_mod_post,...,distinguished,removed_by,updated_utc,og_description,og_title,gilded,rte_mode,author_id,brand_safe,author_created_utc


In [256]:
df[df["whitelist_status"] == "no_ads"].title.value_counts()

Issue: Unstructured text processing    1
Name: title, dtype: int64

In [281]:
df[(df["is_robot_indexable"] == False) & (~df.removed_by_category.isnull())]["selftext"].value_counts()

[removed]                                                       575
[deleted]                                                        41
[removed]\n\n[View Poll](https://www.reddit.com/poll/or4ohx)      1
[removed]\n\n[View Poll](https://www.reddit.com/poll/oliz8o)      1
[removed]\n\n[View Poll](https://www.reddit.com/poll/o0ck41)      1
[deleted]\n\n[View Poll](https://www.reddit.com/poll/lx4fi1)      1
[removed]\n\n[View Poll](https://www.reddit.com/poll/il1hkp)      1
[removed]\n\n[View Poll](https://www.reddit.com/poll/hvf5i6)      1
Name: selftext, dtype: int64

In [296]:
robot_ids = df[df["is_robot_indexable"] == False].id.values

In [319]:
len(robot_ids)

1169

In [297]:
removed_ids = df[(df.selftext.str.startswith("[removed]") == True) | (~df.removed_by_category.isnull())].id.values

In [339]:
df[df.id.isin(list(set(removed_ids) - set(robot_ids)))].full_link.iloc[4]

IndexError: single positional indexer is out-of-bounds

In [320]:
len(removed_ids)

1150

In [300]:
missing_ids = list(set(robot_ids) - set(removed_ids))

In [310]:
df[df.id.isin(missing_ids)].iloc[2].index

Index(['allow_live_comments', 'author', 'author_flair_css_class',
       'author_flair_text', 'author_flair_type', 'author_fullname',
       'author_is_blocked', 'author_patreon_flair', 'author_premium',
       'can_mod_post', 'contest_mode', 'created_utc', 'domain', 'full_link',
       'id', 'is_created_from_ads_ui', 'is_crosspostable', 'is_meta',
       'is_original_content', 'is_reddit_media_domain', 'is_robot_indexable',
       'is_self', 'is_video', 'link_flair_background_color',
       'link_flair_template_id', 'link_flair_text', 'link_flair_text_color',
       'link_flair_type', 'locked', 'media_only', 'no_follow', 'num_comments',
       'num_crossposts', 'over_18', 'parent_whitelist_status', 'permalink',
       'pinned', 'retrieved_on', 'score', 'selftext', 'send_replies',
       'spoiler', 'stickied', 'subreddit', 'subreddit_id',
       'subreddit_subscribers', 'subreddit_type', 'thumbnail', 'title',
       'total_awards_received', 'upvote_ratio', 'url', 'whitelist_status',
  

In [318]:
df.author_premium.value_counts(dropna=False)

False    7962
NaN      1042
True      114
Name: author_premium, dtype: int64

In [325]:
df.thumbnail.value_counts(dropna=False)

self                                                                                6640
default                                                                              634
https://b.thumbs.redditmedia.com/eVEGvPPkUW1Jr0QcdFsz9VlJCeFmVHBBsiD34fDg_ro.jpg      10
https://b.thumbs.redditmedia.com/0d9zYXCdZSUm71X_UkCsCDl97HwhT9TKLRCM3PX-_Fg.jpg       8
spoiler                                                                                7
                                                                                    ... 
https://b.thumbs.redditmedia.com/zVrKx4kej3VgdWLr37EprjBSpjqtofCMy_68zhRcuYg.jpg       1
https://b.thumbs.redditmedia.com/evmwQYHfwLGnN-fnjyu-AM-XG-e24VFWz3GQCtCYLmY.jpg       1
https://b.thumbs.redditmedia.com/5_J6OqyEz-BQi9vic9z3rCC12MW9etN3zPpnnb8W4iw.jpg       1
https://a.thumbs.redditmedia.com/E_HFNdttns3-x7KHFdVq3-q5rfhGNt_DPZvGOWuwH64.jpg       1
http://b.thumbs.redditmedia.com/waDTLBseemU-PBkFkuCbW-LWKZF4dLFoHObkiFiTfjs.jpg        1
Name: thumbnail, Leng

In [329]:
df.upvote_ratio.value_counts(dropna=False)

1.00    7190
NaN     1573
0.50      57
0.99      47
0.67      36
0.75      19
0.81      13
0.33      12
0.86      11
0.60      10
0.83      10
0.84      10
0.94       8
0.96       7
0.89       7
0.92       7
0.93       7
0.98       7
0.91       6
0.25       6
0.80       5
0.66       5
0.88       5
0.78       5
0.87       4
0.95       4
0.97       4
0.76       4
0.72       3
0.90       3
0.82       3
0.71       3
0.57       3
0.79       2
0.40       2
0.85       2
0.20       2
0.46       1
0.73       1
0.52       1
0.21       1
0.70       1
0.53       1
0.31       1
0.38       1
0.44       1
0.35       1
0.55       1
0.62       1
0.17       1
0.29       1
0.63       1
0.74       1
Name: upvote_ratio, dtype: int64

In [330]:
pd.DataFrame(df[df.id.isin(missing_ids)].iloc[2]).iloc[70:80]

,8157
og_title,None
gilded,NaN
rte_mode,None
author_id,None
brand_safe,None
author_created_utc,NaN


In [288]:
df[df.selftext.str.startswith("[removed]").fillna(False)]["is_robot_indexable"].value_counts(dropna=False)

False    660
NaN        4
Name: is_robot_indexable, dtype: int64

In [289]:
df[df.selftext.str.startswith("[removed]").fillna(False)]["removed_by_category"].value_counts(dropna=False)

reddit              416
automod_filtered     99
NaN                  84
moderator            65
Name: removed_by_category, dtype: int64

In [342]:
df["removed_by"].value_counts(dropna=False)

NaN       9108
reddit      10
Name: removed_by, dtype: int64

In [355]:
df["brand_safe"].value_counts()

False    117
Name: brand_safe, dtype: int64

In [354]:
df[(df.total_awards_received.isnull()) & (df.gilded.isnull())][["total_awards_received", "gilded"]]

,total_awards_received,gilded
540,NaN,NaN
1705,NaN,NaN
2219,NaN,NaN
4092,NaN,NaN
4130,NaN,NaN
...,...,...
9024,NaN,NaN
9025,NaN,NaN
9026,NaN,NaN
9027,NaN,NaN


In [352]:
df.shape

(9118, 76)